# Hyperparameter Tuning using Your Own Keras/Tensorflow Container - Plant Seedling Classification

This notebook shows how to build your own Keras(Tensorflow) container and use SageMaker for training, leveraging hyperparameter tuning. 

We use VGG19 with Imagenet weights to perform transfer learning using Keras and train a plant seedling classification model

This tutorial uses code from https://github.com/awslabs/amazon-sagemaker-examples/tree/master/hyperparameter_tuning/keras_bring_your_own

## Set up the notebook instance to support local mode
Currently you need to install docker-compose in order to use local mode (i.e., testing the container in the notebook instance without pushing it to ECR).

In [62]:
!/bin/bash setup.sh

## Set up the environment
We will set up a few things before starting the workflow. 

1. get the execution role which will be passed to sagemaker for accessing your resources such as s3 bucket
2. specify the s3 bucket and prefix where training data set and model artifacts are stored

In [1]:
import os
import numpy as np
import tempfile

import tensorflow as tf

import sagemaker
import boto3
from sagemaker.estimator import Estimator

region = boto3.Session().region_name

sagemaker_session = sagemaker.Session()
smclient = boto3.client('sagemaker')

bucket = sagemaker.Session().default_bucket()  # s3 bucket name, must be in the same region as the one specified above
prefix = 'sagemaker/hpo-keras-seedling'

role = sagemaker.get_execution_role()

# Upload Data to S3

In [ ]:
sagemaker_session.upload_data(path='npX_keras.pkl', bucket=bucket, key_prefix='data/hpo-keras-seedling/train')
sagemaker_session.upload_data(path='oh_npY.pkl', bucket=bucket, key_prefix='data/hpo-keras-seedling/train')

In [3]:
!aws s3 ls s3://<bucket>/data/hpo-keras-seedling/train --recursive --human

2018-08-01 04:16:52  317.3 MiB data/hpo-keras-seedling/train/npX_keras.pkl
2018-08-01 04:16:55  876.9 KiB data/hpo-keras-seedling/train/oh_npY.pkl


## Complete source code
- [trainer/start.py](trainer/start.py): Keras model
- [trainer/environment.py](trainer/environment.py): Contain information about the SageMaker environment

## Building the image
We will build the docker image using the Tensorflow versions on dockerhub. The full list of Tensorflow versions can be found at https://hub.docker.com/r/tensorflow/tensorflow/tags/


In [4]:
import shlex
import subprocess

def get_image_name(ecr_repository, tensorflow_version_tag):
    return '%s:tensorflow-%s' % (ecr_repository, tensorflow_version_tag)

def build_image(name, version):
    cmd = 'docker build -t %s --build-arg VERSION=%s -f Dockerfile .' % (name, version)
    subprocess.check_call(shlex.split(cmd))

#version tag can be found at https://hub.docker.com/r/tensorflow/tensorflow/tags/ 
#e.g., latest cpu version is 'latest', while latest gpu version is 'latest-gpu'
tensorflow_version_tag = 'latest-gpu'   

account = boto3.client('sts').get_caller_identity()['Account']
    
ecr_repository="%s.dkr.ecr.%s.amazonaws.com/test" %(account,region) # your ECR repository, which you should have been created before running the notebook

image_name = get_image_name(ecr_repository, tensorflow_version_tag)

print('building image:'+image_name)
build_image(image_name, tensorflow_version_tag)

building image:127230316473.dkr.ecr.us-west-2.amazonaws.com/test:tensorflow-latest-gpu


In [5]:
# channels
train_data_location = 's3://sagemaker-us-west-2-127230316473/data/hpo-keras-seedling/train'
channels = {'train': train_data_location}

# ECR Setup

## ADD IAM role allowing access to ECR (attach 1 policies) 

In [20]:

## refer to tutorial link below to grant SageMaker IAM access to ECR:
#https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb
        

In [ ]:
# create our own ECR repository if it doesn't exist

In [6]:
# !aws ecr create-repository --repository-name test

## Pushing the container to ECR
Now that we've tested the container locally and it works fine, we can move on to run the hyperparmeter tuning. Before kicking off the tuning job, you need to push the docker image to ECR first. The ECR repository has been set up in the beginning of the sample notebook, please make sure you have input your ECR repository information there.

In [7]:
def push_image(name):
    cmd = 'aws ecr get-login --no-include-email --region '+region
    login = subprocess.check_output(shlex.split(cmd)).strip()

    subprocess.check_call(shlex.split(login.decode()))

    cmd = 'docker push %s' % name
    subprocess.check_call(shlex.split(cmd))

print ("pushing image:"+image_name)
push_image(image_name)

pushing image:127230316473.dkr.ecr.us-west-2.amazonaws.com/test:tensorflow-latest-gpu


## Specify hyperparameter tuning job configuration
*Note, with the default setting below, the hyperparameter tuning job can take 20~30 minutes to complete. You can customize the code in order to get better result, such as increasing the total number of training jobs, epochs, etc., with the understanding that the tuning time will be increased accordingly as well.*

Now you configure the tuning job by defining a JSON object that you pass as the value of the TuningJobConfig parameter to the create_tuning_job call. In this JSON object, you specify:
* The ranges of hyperparameters you want to tune
* The limits of the resource the tuning job can consume 
* The objective metric for the tuning job

In [9]:
import json
from time import gmtime, strftime

tuning_job_name = 'seedling-keras-HPO-' + strftime("%d-%H-%M-%S", gmtime())

print(tuning_job_name)

tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "0.0003",
          "MinValue": "0.0001",
          "Name": "learning_rate",          
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "300",
          "MinValue": "150",
          "Name": "epochs",          
        },
        {
          "MaxValue": "36",
          "MinValue": "24",
          "Name": "batch_size",          
        }
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 8,
      "MaxParallelTrainingJobs": 1
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "acc",
      "Type": "Maximize"
    }
  }


seedling-keras-HPO-03-01-12-17


## Specify training job configuration
Now you configure the training jobs the tuning job launches by defining a JSON object that you pass as the value of the TrainingJobDefinition parameter to the create_tuning_job call.
In this JSON object, you specify:
* Metrics that the training jobs emit
* The container image for the algorithm to train
* The input configuration for your training and test data
* Configuration for the output of the algorithm
* The values of any algorithm hyperparameters that are not tuned in the tuning job
* The type of instance to use for the training jobs
* The stopping condition for the training jobs

This example defines one metric that Tensorflow container emits: loss. 

In [11]:
training_image = image_name


training_job_definition = {
    "AlgorithmSpecification": {
      "MetricDefinitions": [
        {
          "Name": "acc",
          "Regex": "accuracy: ([0-9\\.]+)"
        }
      ],
      "TrainingImage": training_image,
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": channels['train'],
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        },
        {
            "ChannelName": "test",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": channels['train'],
                    "S3DataDistributionType": "FullyReplicated"
                }
            },            
            "CompressionType": "None",
            "RecordWrapperType": "None"            
        }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/{}/output".format(bucket,prefix)
    },
    "ResourceConfig": {
      "InstanceCount": 1,
      "InstanceType": "ml.p2.xlarge",
      "VolumeSizeInGB": 50
    },
    "RoleArn": role,
    "StaticHyperParameters": {
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 43200
    }
}


## Create and launch a hyperparameter tuning job
Now you can launch a hyperparameter tuning job by calling create_tuning_job API. Pass the name and JSON objects you created in previous steps as the values of the parameters. After the tuning job is created, you should be able to describe the tuning job to see its progress in the next step, and you can go to SageMaker console->Jobs to check out the progress of each training job that has been created.

In [12]:
len(tuning_job_name)

30

In [13]:
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = tuning_job_name,
                                               HyperParameterTuningJobConfig = tuning_job_config,
                                               TrainingJobDefinition = training_job_definition)

{'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-west-2:127230316473:hyper-parameter-tuning-job/seedling-keras-hpo-03-01-12-17',
 'ResponseMetadata': {'RequestId': 'c4435e2b-75d9-4954-ad13-e56cef30c760',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c4435e2b-75d9-4954-ad13-e56cef30c760',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '131',
   'date': 'Fri, 03 Aug 2018 01:12:54 GMT'},
  'RetryAttempts': 0}}

Let's just run a quick check of the hyperparameter tuning jobs status to make sure it started successfully and is `InProgress`.

In [15]:
smclient.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName = tuning_job_name)['HyperParameterTuningJobStatus']

'Completed'

In [19]:
4.832214765100671 * 298

1440.0

In [16]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
full_df = tuner.dataframe()
full_df

,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,batch_size,epochs,learning_rate
0,0.767361,1463.0,2018-08-03 04:36:00+00:00,seedling-keras-HPO-03-01-12-17-008-c744b7c3,Completed,2018-08-03 04:11:37+00:00,36.0,298.0,0.000300
1,0.827559,1477.0,2018-08-03 04:07:43+00:00,seedling-keras-HPO-03-01-12-17-007-0554dfc8,Completed,2018-08-03 03:43:06+00:00,24.0,288.0,0.000299
2,0.807978,1486.0,2018-08-03 03:37:20+00:00,seedling-keras-HPO-03-01-12-17-006-f74e33fe,Completed,2018-08-03 03:12:34+00:00,28.0,300.0,0.000271
3,0.837662,1586.0,2018-08-03 03:09:22+00:00,seedling-keras-HPO-03-01-12-17-005-c7e1ad7c,Completed,2018-08-03 02:42:56+00:00,28.0,300.0,0.000296
4,0.816444,1506.0,2018-08-03 02:38:43+00:00,seedling-keras-HPO-03-01-12-17-004-dddd668c,Completed,2018-08-03 02:13:37+00:00,29.0,296.0,0.000280
5,0.616898,879.0,2018-08-03 02:11:41+00:00,seedling-keras-HPO-03-01-12-17-003-ac360a4f,Completed,2018-08-03 01:57:02+00:00,36.0,170.0,0.000172
6,0.653935,1058.0,2018-08-03 01:52:56+00:00,seedling-keras-HPO-03-01-12-17-002-37c6d555,Completed,2018-08-03 01:35:18+00:00,36.0,201.0,0.000221
7,0.635158,1093.0,2018-08-03 01:33:01+00:00,seedling-keras-HPO-03-01-12-17-001-ad5afbb8,Completed,2018-08-03 01:14:48+00:00,25.0,196.0,0.000119
